# Pointers
One of the biggest hurdles when transitioning from Python to Cython is the use of pointers in C code, which is necessary to write clean Cython programs.


In C, every variable has a memory location and every memory location has a memory address that can be accessed using the `&` symbol.

In [1]:
# enable Cython in the IPython Noteboook
%load_ext Cython

In [2]:
%%cython
# from libc.stdio cimport printf

# this works, but unfortunately prints to the command line and not the notebook
# https://github.com/ipython/ipython/issues/1230
# cdef int variable_1 = 10
# cdef int variable_2 = 20
# printf("Address of variable_1: %x\n", &variable_1);
# printf("Address of variable_2: %x\n", &variable_2);

# this throws an error, is there any way I can fix this?
cdef int variable_1 = 10
cdef int variable_2 = 20
print &variable_1
print &variable_2


Error compiling Cython file:
------------------------------------------------------------
...
# printf("Address of variable_2: %x\n", &variable_2);

# this throws an error, is there any way I can fix this?
cdef int variable_1 = 10
cdef int variable_2 = 20
print &variable_1
     ^
------------------------------------------------------------

/Users/nelsonliu/.ipython/cython/_cython_magic_e74bf1eb9fd67c9f5ac2f4184431bed8.pyx:13:6: Cannot convert 'int *' to Python object

Error compiling Cython file:
------------------------------------------------------------
...

# this throws an error, is there any way I can fix this?
cdef int variable_1 = 10
cdef int variable_2 = 20
print &variable_1
print &variable_2
     ^
------------------------------------------------------------

/Users/nelsonliu/.ipython/cython/_cython_magic_e74bf1eb9fd67c9f5ac2f4184431bed8.pyx:14:6: Cannot convert 'int *' to Python object


# So what exactly are pointers?
A pointer is a variable whose value is the memory address of another variable. Changing the pointer changes the value at the memory address.

To declare a pointer, use the following syntax:

In [3]:
%%cython
cdef int* integer_pointer # declares a pointer to an integer
cdef double* double_pointer # declares a pointer to a double
cdef float* float_pointer # declares a pointer to a float
cdef char* char_pointer # declares a pointer to a char

While these pointers all point to memory addresses, the variable or constant at the memory address the pointer is pointing to are all of different types.

# How do I use pointers?
Pointers are generally used to modify the value of a variable or constant that it points to. In C, this is done through use of the `*` operator, which returns the value of the variable or constant specified by the pointer type. However, the `*` operator already has a meaning in Python -- unpacking tuples. As a result, assigning and accessing pointers uses an index notation similar to lists.

In [4]:
%%cython
# define a variable to modify
cdef int to_increment = 0
# define an int pointer with the * operator
cdef int* incrementor

# store the address of the variable in the pointer
incrementor = &to_increment

# access the value at the pointer by using indexing notation and 
# increment the value the pointer points at.
incrementor[0] += 1
print "Value of variable set to pointer: {}".format(incrementor[0])
print "Value of original variable: {}".format(to_increment)

# increment the original variable
to_increment += 1
print "Value of variable set to pointer: {}".format(incrementor[0])
print "Value of original variable: {}".format(to_increment)

Value of variable set to pointer: 1
Value of original variable: 1
Value of variable set to pointer: 2
Value of original variable: 2


As you can see, modifying the value of what the pointer was pointing at (by adding one) changed the value of the original variable, and changes to the original variable propogated to affect the value that the pointer was pointing at.